In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L4\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['Capital.csv', 'Central.csv', 'Dunwoody.csv', 'Genese.csv', 'Hudson.csv', 'LongIsland.csv', 'Milwood.csv', 'NYC.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[2]))

,Demand,Normalized_net,Temprature
0,643.8,0.467026,-5.8
1,612.9,0.444610,-6.1
2,587.9,0.426475,-7.5
3,570.0,0.413490,-8.4
4,563.5,0.408774,-7.4
...,...,...,...
43819,696.4,0.505183,3.4
43820,668.5,0.484944,3.3
43821,637.9,0.462746,2.6
43822,608.4,0.441346,2.7


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[2]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temprature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_Dunwoody,Temp_Dunwoody
0,643.8,-5.8
1,612.9,-6.1
2,587.9,-7.5
3,570.0,-8.4
4,563.5,-7.4
...,...,...
43819,696.4,3.4
43820,668.5,3.3
43821,637.9,2.6
43822,608.4,2.7


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_Dunwoody,Temp_Dunwoody,Lag-2,Lag-7
0,690.0,-13.9,604.8,643.8
1,662.3,-14.2,584.8,612.9
2,646.5,-14.8,569.3,587.9
3,640.1,-15.6,544.0,570.0
4,645.9,-16.0,559.3,563.5
...,...,...,...,...
43651,696.4,3.4,687.7,693.1
43652,668.5,3.3,678.1,672.7
43653,637.9,2.6,654.5,655.1
43654,608.4,2.7,616.6,630.8


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 15s 9ms/step - loss: 0.0116 - mape: 5970.7881 - mae: 0.0821 - val_loss: 0.0060 - val_mape: 36.1355 - val_mae: 0.0614
Epoch 2/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0070 - mape: 5279.6831 - mae: 0.0640 - val_loss: 0.0070 - val_mape: 36.1349 - val_mae: 0.0626
Epoch 3/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0050 - mape: 4398.9448 - mae: 0.0542 - val_loss: 0.0080 - val_mape: 36.0545 - val_mae: 0.0650
Epoch 4/100
1105/1105 [==============================] - 9s 9ms/step - loss: 0.0045 - mape: 4237.0898 - mae: 0.0511 - val_loss: 0.0071 - val_mape: 32.4921 - val_mae: 0.0615
Epoch 5/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0041 - mape: 4313.9248 - mae: 0.0485 - val_loss: 0.0092 - val_mape: 27.6452 - val_mae: 0.0674
Epoch 6/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0038 - mape: 3735.3857 - mae: 0.0469 - val_loss: 0.0070

1105/1105 [==============================] - 9s 8ms/step - loss: 0.0020 - mape: 1286.2599 - mae: 0.0335 - val_loss: 0.0078 - val_mape: 27.7889 - val_mae: 0.0607
Epoch 49/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0020 - mape: 1558.9640 - mae: 0.0334 - val_loss: 0.0074 - val_mape: 27.3807 - val_mae: 0.0585
Epoch 50/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0019 - mape: 1447.4261 - mae: 0.0331 - val_loss: 0.0075 - val_mape: 25.8842 - val_mae: 0.0591
Epoch 51/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0019 - mape: 1641.8143 - mae: 0.0330 - val_loss: 0.0070 - val_mape: 26.2746 - val_mae: 0.0564
Epoch 52/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0019 - mape: 1306.2633 - mae: 0.0325 - val_loss: 0.0077 - val_mape: 26.7916 - val_mae: 0.0588
Epoch 53/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0018 - mape: 1192.0323 - mae: 0.0324 - val_loss: 0.0078 - val_m

1105/1105 [==============================] - 8s 7ms/step - loss: 0.0011 - mape: 1193.4943 - mae: 0.0248 - val_loss: 0.0080 - val_mape: 29.8584 - val_mae: 0.0631
Epoch 96/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0011 - mape: 755.7092 - mae: 0.0248 - val_loss: 0.0075 - val_mape: 27.4308 - val_mae: 0.0606
Epoch 97/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0011 - mape: 1165.5835 - mae: 0.0248 - val_loss: 0.0077 - val_mape: 29.6864 - val_mae: 0.0622
Epoch 98/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0010 - mape: 907.2338 - mae: 0.0244 - val_loss: 0.0078 - val_mape: 28.8810 - val_mae: 0.0619
Epoch 99/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0010 - mape: 758.8801 - mae: 0.0243 - val_loss: 0.0075 - val_mape: 28.5428 - val_mae: 0.0605
Epoch 100/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0010 - mape: 965.6078 - mae: 0.0242 - val_loss: 0.0079 - val_mape

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 875.6800487041473


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 5ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.19199145, 0.16926467, 0.1589391 , ..., 0.33064115, 0.2826948 ,
       0.23839688], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.39619938, 0.36517685, 0.34248445, ..., 0.29087599, 0.26263017,
       0.23524609])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,747.9,534.624756
1,715.5,510.888855
2,691.8,500.104797
3,680.0,492.621765
4,699.9,509.379242
...,...,...
4363,696.4,749.399597
4364,668.5,726.371948
4365,637.9,679.430786
4366,608.4,629.355530


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

8.490245476867301